# 5. 闭包与装饰器
闭包：   
   闭包是词法闭包（Lexical Closure）的简称，是引用了自由变量的函数，这个被引用的自由变量将和这个函数一同存在，即使已经离开了创造它的环境也不例外。   
   3个条件：有内嵌函数  ；内嵌引用自由变量  ；  外部函数返回内嵌函数
   
   https://www.cnblogs.com/wxzhao/p/6297551.html

In [1]:
#一个例子
def outer(age):
    a = {}
    def get_info(name=None, work=None, age=age):
        if age < 18:
            a['name'] = name
            a['age'] = age
        else:
            a['name'] = name
            a['age'] = age
            a['work'] = work
        return a
    return get_info

if __name__ == "__main__":
    p1 = outer(15)
    p2 = outer(20)

    print(p1('zhangsan', None))
    print(p1('shenzhao', 'xiwangong'))
    print(p2('lisi', 'pythoner'))
    print(p2('shenzhao', 'xiwangong'))
    
    
#按照函数的生命周期分析，在执行完p1=outer（15）之后，
#返回了一个函数对象赋值给p1，正常情况下局部变量age应该销毁了，
#但是为什么再执行p1()的时候还能访问到age呢？

#这涉及到一个特殊的定义:自由变量， 
#自由变量的定义很简单，闭包所引用的外部作用域变量称为自由变量。  age 和  a 都是自由变量

#当一个变量变成自由变量之后，其作用域会发生相应的改变，自由变量随闭包存活

{'name': 'zhangsan', 'age': 15}
{'name': 'shenzhao', 'age': 15}
{'name': 'lisi', 'age': 20, 'work': 'pythoner'}
{'name': 'shenzhao', 'age': 20, 'work': 'xiwangong'}


In [2]:
def counter(start=0):
    count = start
    def incr():
        count += 1
        return count
    return incr

a = counter()
print('a:',a())

#只能改变可变对象 不能改变不可变对象   把count换成list  这个就能执行

UnboundLocalError: local variable 'count' referenced before assignment

In [4]:
def foo():
    a = []
    def bar():
        a.append(100)
        return a
    return bar

if __name__ == "__main__":
    b = foo()
    c = foo()

    print(b())
    print(b())
    print(c())
    
#b = foo()跟c = foo()返回的是两个不同的对象，也就有两个自由变量， 分别是b 的a[]  和  c的 a[]

#自由变量的生命周期等同于闭包的生命周期，也就是说在第二次执行print b()的时候，自由变量是被第一次执行print b()操作之后的。。。
#就是b的第二个append100也加入到b的那个自由变量a[]里了  先加100  再加入100

[100]
[100, 100]
[100]


装饰器:   
python装饰器就是用于拓展原来函数功能的一种函数，这个函数的特殊之处在于它的返回值也是一个函数，使用python装饰器的好处就是在不用更改原函数的代码前提下给函数增加新的功能。

装饰器就是一种的闭包的应用，只不过其传递的是函数：


https://blog.csdn.net/xiangxianghehe/article/details/77170585

In [1]:
#普通装饰器
import time

def deco(func):
    def wrapper():
        startTime = time.time()
        func()
        endTime = time.time()
        msecs = (endTime - startTime)*1000
        print("time is %d ms" %msecs)
    return wrapper


@deco
def func2():
    print("hello")
    time.sleep(1)
    print("world")

if __name__ == '__main__':
    f = func2 #这里f被赋值为func，执行f()就是执行func()
    func2()

hello
world
time is 1000 ms


In [6]:
#带有参数的装饰器
import time

def deco(func):
    def wrapper(a,b):
        startTime = time.time()
        func(a,b)
        endTime = time.time()
        msecs = (endTime - startTime)*1000
        print("time is %d ms" %msecs)
    return wrapper


@deco  # func(a,b)=deco(func(a,b))
def func(a,b):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b))

if __name__ == '__main__':
    f = func
    func(3,4)
    #func()

hello，here is a func for add :
result is 7
time is 1009 ms


In [7]:
#带有不定参的装饰器
import time

def deco(func):
    def wrapper(*args, **kwargs):
        startTime = time.time()
        func(*args, **kwargs)
        endTime = time.time()
        msecs = (endTime - startTime)*1000
        print("time is %d ms" %msecs)
    return wrapper


@deco
def func(a,b):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b))

@deco
def func2(a,b,c):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b+c))


if __name__ == '__main__':
    f = func
    func2(3,4,5)
    f(3,4)
    #func()

hello，here is a func for add :
result is 12
time is 1009 ms
hello，here is a func for add :
result is 7
time is 1009 ms


In [8]:
#多个装饰器
import time

def deco01(func):
    def wrapper(*args, **kwargs):
        print("this is deco01")
        startTime = time.time()
        func(*args, **kwargs)
        endTime = time.time()
        msecs = (endTime - startTime)*1000
        print("time is %d ms" %msecs)
        print("deco01 end here")
    return wrapper

def deco02(func):
    def wrapper(*args, **kwargs):
        print("this is deco02")
        func(*args, **kwargs)

        print("deco02 end here")
    return wrapper

@deco01    #func=deco01(deco02(func（a,b）))       <--------step2
@deco02   #func=deco02(func（a,b）)              <--------step1
def func(a,b):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b))



if __name__ == '__main__':
    f = func
    f(3,4)
    #func()

this is deco01
this is deco02
hello，here is a func for add :
result is 7
deco02 end here
time is 1009 ms
deco01 end here


## 6  递归的妙用  重启运行失败的函数

In [26]:
import time

count = 0

def compute_number():
    print('compute_number')
    lis=[1,2,'a']
    lis=[float(i) for i in lis]

def main():  
   

    global count

    if count < 3:
        try:
            count += 1
            compute_number()
        except Exception as e:
            print(e)
            main()
        finally:
            print('final')

if __name__ == "__main__":  
    main()

compute_number
could not convert string to float: 'a'
compute_number
could not convert string to float: 'a'
compute_number
could not convert string to float: 'a'
final
final
final
